# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [1]:
# Example reading the SQL database from here

from utils import db_connect
import pandas as pd
import matplotlib.pyplot as plt

#engine = db_connect()

#dataframe = pd.read_sql("Select * from books;", engine)
#print(dataframe.describe())

In [2]:
# Example importing the CSV here

df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/linear-regression-project-tutorial/main/medical_insurance_cost.csv')


# dataframe = pd.read_csv('../path/to/file.csv')
# dataframe.describe()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
df.sample(5)

,age,sex,bmi,children,smoker,region,charges
1193,48,female,36.575,0,no,northwest,8671.19125
829,39,male,21.850,1,no,northwest,6117.49450
655,52,female,25.300,2,yes,southeast,24667.41900
412,26,female,17.195,2,yes,northeast,14455.64405
955,31,male,39.490,1,no,southeast,3875.73410


In [6]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010
